In [ ]:
Grid search
  가능한 모든 조합 > 가장 높은 성능 하이퍼파라미터 조합 > 최적조합 선정
  체계적+균등한 전역적인 탐색
  탐색 기간이 매우 김
  예측 가능 > .predict_proba
  주요 파라미터
    첫 번째 인수 : n_estimator > 모델 객체
    두 번째 인수 : param_grid > 하이퍼파라미터 딕셔너리만들기 > 'key' : 하이퍼파라미터명, 'value' : 탐색구간(딕셔너리, iterable객체)
    세 번째 인수 : scoring = '평가 지표'
    네 번째 인수 : cv = cv or 정수
from sklearn.model_selection import GridSearchCV
hp = {

}
grid_Search = GridSearchCV(model, hp, scoring = '평가 지표', cv = cv, n_jobs = -1)
grid_search.fit(train_ft, target)
pred = grid_search.predict_proba(test_ft)[1]

최적 하이퍼파라미터 조합 cv 점수
  grid_search.best_scroe_

최적 하이퍼파라미터 조합
  grid_search.best_params_

※ 탐색한 재조합 다시 모델파라미터에 적용해 학습시키기
model = 모델함수 or 클래스(파라미터 설정, **최적조합)
model.fit(train_ft, target) > 최적의 파라미터 조합 재학습
pred = model.predict_proba(test_ft)[1] > 재학습으로 인한 예측

Random search
  지정한 횟수만큼 하이퍼파라미터 조합 > 랜덤 샘플링 > 최적조합 선정
  주요파라미터
    첫 번째 인수 : estimator : 모델 객체
    두 번째 인수 : grid와 동일
    세 번째 인수 : n_iter = n > 탐색 횟수(default= 10)
    네 번째 인수 : scoring = '평가지표'
    다섯 번째 인수 : cv = cv객체 or 정수값
    여섯 번째 인수 : random_state = 42
from sklearn.model_selection import RandomizedSearchCV
hp = {

}
rand_search = RandomizedSearchCV(model, hp, cv = cv, scroing = '평가지표', n_iter = n, n_jobs = -1, random_state = 42)
rand_search.fit(train_ft, target)
pred = rand_search.predict(test_ft)

# 최적 조합 점수
rand_search.best_score_
# 최적 조합
rand_search.best_params_

다수 모델 반복문 튜닝
  모델 선언
from 모델객체

[모델클래스, {하이퍼파라미터 탐색범위}]
tuning_models = [
    [모델객체1, {'파라미터key', '탐색범위value'}],
    [모델객체2, {'파라미터key', '탐색범위value'}],
    etc
]

반복문
for model_hp in tuning_models:
  model_cls, hp = model_hp
  rand_search = RandomizedSearchCV(model_cls(), hp, cv = cv, scoring = '평가지표', random_state = 42, n_jobs = -1, n_iter = n)
  rand_search.fit(train_ft, target)
  model_hp.append(rand_search)

모델별 파라미터 조합 확인하기
for model_cls, _, search_obj in tuning_models:
  print(model_cls.__name__, search_obj.best_score_, search_obj.best_params_)

예측하기
pred_list = []
for _, _, search_obj in tuning_models:
  pred = search_obj.predict_proba(test.ft)[1]
  pred_list.append(pred)

pred = np.array(pred_list).mean(axis = 0)
====================================================================================================================================
베이지안 최적화
  매번 새로운 하이퍼파라미터 조합
  최대 or 최소인 최적의 CV값 탐색
  optuna
optuna모델 튜닝
  1.목적함수(콜백) 만들기 > 직접 제작
    1-1. optina 라이브러리 > 목적함수에 trial객체 전달
    1-2. tiral객체 suggest메서드 > 사전지식 반영 > 하이퍼파라미터 제안+세팅
    1-3. 모델학습 및 검증평가 결과 반환
  2. 대체모델역할 sampler객체 생성
  3. 전체적 튜닝 study객체 생성 > optimize메서드 실행 > 튜닝시작(파라미터 : n_trial = n)
  optuna.trial.Trial > trial.suggest_int, trial.suggest_float, trial.suggest_categorical
  첫 번째 인수 : 해당 하이퍼파라미터명
  두 번째 인수 : 탐색 범위

%pip install optuna
import optuna
# ex)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, KFold
# 전역 변수 사용 > 비추천
def objective(trial):
  hp = {
      '파라미터명1' : trial.suggest_int('파라미터명1', 탐색범위),
      '파라미터명2' : trial.suggest_float('파라미터명2', 탐색범위),
      '파라미터명3' : trial.suggest_categorical('파라미터명3', 탐색범위)
  }
  cv = KFold(k, shuffle = True, random_state = 42)
  model = RandomForestClassifier(**hp, random_state = 42)
  score = cross_val_score(model, train_ft, target, cv = cv, scoring = '평가지표', n_jobs = -1).mean()
  return score

# 대체모델역할을 하는 샘플러 객체
sampler = optuna.samplers.TPESampler(seed=SEED)

# 스터디 객체
study = optuna.create_study(
    direction="maximize", # '1'에 가까울수록 좋은 경우 : maximize, '0'에 가까울수록 좋은경우 : minimize
    sampler = sampler
)
study.optimize(objective, n_trials=50)
study.best_value
study.best_trial.params

# lambda함수 사용
def objective(trial,x,y):
  hp = {
      동일
  }
  cv = KFold(k, shuffle = True, random_state = 42)
  model = RandomForestClassifier(**hp, random_state = 42)
  score = cross_val_score(model, x, y, cv = cv, scoring = '평가지표', n_jobs = -1).mean()
  return score

# 대체모델역할을 하는 샘플러 객체
sampler = optuna.samplers.TPESampler(seed=SEED)

# 스터디 객체
study = optuna.create_study(
    direction="maximize", # '1'에 가까울수록 좋은 경우 : maximize, '0'에 가까울수록 좋은경우 : minimize
    sampler = sampler
)
study.optimize(objective, n_trials=50)
study.best_value
study.best_trial.params

# 클래스 사용
class Objective:
  def __init__(self, x,y, seed): # seed > 전역변수 선언한 경우
    self.x = x
    self.y = y
    self.seed = seed
    self.cv = KFold(k, shuffle = True, random_state = seed)
  def __call__(self, trial):
    hp = {
      동일
  }
  model = RandomForestClassifier(**hp, random_state = seed)
  score = cross_val_score(model, self.x, self.y, self.cv = cv, scoring = '평가지표', n_jobs = -1)

# 대체모델역할을 하는 샘플러 객체
sampler = optuna.samplers.TPESampler(seed=SEED)

# 스터디 객체
study = optuna.create_study(
    direction="maximize", # '1'에 가까울수록 좋은 경우 : maximize, '0'에 가까울수록 좋은경우 : minimize
    sampler = sampler
)
study.optimize(objective, n_trials=50)
study.best_value
study.best_trial.params